# OpenCV Foreground Extraction

In [ ]:
!pip install opencv-python
import cv2

In [ ]:
# download the image file we'll be using
import sys
import types
import pandas as pd
import numpy as np
import urllib.request

img_name = 'BSDSImage.png'

url = 'https://github.com/jacquesroy/byte-size-data-science/raw/master/data/' + img_name
# filename = url.rsplit('/', 1)[-1]
urllib.request.urlretrieve(url, img_name)

%ls -l

## Display the image
The image we are uzing has a  width of 864 and a height: 480<br/>
The OpenCV library expects that the width and height are multiples of 32.

- Width : 864 --> 27 * 32
- Height: 480 --> 15 * 32

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 10))
img = plt.imread(img_name)
plt.axis('off')
plt.imshow(img)

## Get the model weights
This particular model is trained on COCO dataset (common objects in context) from Microsoft.
It is capable of detecting 80 common objects:

airplane, apple, backpack, banana, baseball bat, baseball glove, bear, bed, bench, bicycle, bird, boat, book, bottle, bowl, broccoli, bus, 
cake, car, carrot, cat, cell phone, chair, clock, couch, cow, cup, dining table, dog, donut, elephant, fire hydrant, fork, frisbee, giraffe, 
hair drier, handbag, horse, hot dog, keyboard, kite, knife, laptop, microwave, motorcycle, mouse, orange, oven, parking meter, person, 
pizza, potted plant, refrigerator, remote, sandwich, scissors, sheep, sink, skateboard, skis, snowboard, spoon, sports ball, stop sign, 
suitcase, surfboard, teddy bear, tennis racket, tie, toaster, toilet, toothbrush, traffic light, train, truck, tv, umbrella, vase, wine glass, 
zebra

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://github.com/pjreddie/darknet/raw/master/cfg/yolov3.cfg
!ls -l yolo*

In [ ]:
classes = ["person", "bicycle", "car", "motorcycle", "airplane", "bus",
"train", "truck", "boat", "traffic light", "fire hydrant",
"stop sign", "parking meter", "bench", "bird", "cat", "dog",
"horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe",
"backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee",
"skis", "snowboard", "sports ball", "kite", "baseball bat", 
"baseball glove", "skateboard", "surfboard", "tennis racket",
"bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl",
"banana", "apple", "sandwich", "orange", "broccoli", "carrot",
"hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant",
"bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote",
"keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
"refrigerator", "book", "clock", "vase", "scissors", "teddy bear",
"hair drier", "toothbrush"]

In [ ]:
# Read the color image
# options: IMREAD_COLOR (1), IMREAD_GRAYSCALE (0), IMREAD_UNCHANGED (-1)
img = cv2.imread(img_name, cv2.IMREAD_COLOR) # Returns None on bad file
dims = img.shape
print("Image width: {}, height: {}, depth: {}".format(dims[1], dims[0], dims[2]))

### Loading the model
We instantiate the model from the files loader earlier: yolov3.weights yolov3.cfg

Then we take a look at some attributes of the model.

In [ ]:
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

### Process the image
We convert the img numpy array of uint8 and shape (576, 768, 3) to another numpy array of float32 and shape (1, 3, 576, 768).
We then set that blob numpy rray as the input to our model.

The scale value is a multiplier for the values in the array. This way, all the values should be smaller or equal to one. Small values are better for neural networks.

In [ ]:
scale = 1./255
dims = img.shape
# blobFromImage(image, scale, (Width,Height), (0,0,0), True, crop=False)
blob = cv2.dnn.blobFromImage(img, scale, (dims[1], dims[0]), (0,0,0), True, crop=False)

# Set the input to the model
net.setInput(blob)

In [ ]:
# COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color1 = np.array([0.0,0.0,255.]) # red
    color2 = np.array([0.0,255.0,255.0])# Other yellow
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color1, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color2, 2)

In [ ]:
# run inference through the network and gather predictions from output layers
outs = net.forward(get_output_layers(net))
# input image shape (dims=img.shape)
Width=dims[1]
Height=dims[0]

# initialization
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

# for each detection from each output layer, get the confidence, class id, bounding box params
# and ignore weak detections (confidence < 0.5)
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [ ]:
print("Classes found: " + str(class_ids))
print("Classes names: " + str([classes[i] for i in class_ids]) )
print("Classes confidence: " + str(confidences))

In [ ]:
# apply non-max suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
# go through the detections remaining
# after nms and draw bounding box
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    
    draw_bounding_box(img, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

# save output image to disk
retval = cv2.imwrite("object-detection.jpg", img)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

img_plt = plt.imread(img_name)
img2_plt = plt.imread('object-detection.jpg')

fig, ax = plt.subplots(1,2)
ax[0].axis('off')
ax[1].axis('off')
ax[0].figure.set_size_inches(18,10)
ax[1].figure.set_size_inches(18,10)

ax[0].imshow(img_plt);
ax[1].imshow(img2_plt);

In [ ]:
# To see the object detection image bigger
fig = plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(img2_plt)

In [ ]:
# Class 1 person
rect=boxes[0]
rect=(int(boxes[0][0]),int(boxes[0][1]),int(boxes[0][2]),int(boxes[0][3]))

In [ ]:
mask = np.zeros(img.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

# Re-read the original image so we can apply the grabcut on the original 
# and not the one with the red box
img0 = cv2.imread(img_name, cv2.IMREAD_COLOR)

cv2.grabCut(img0,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')

img2 = img*mask2[:,:,np.newaxis]
retval = cv2.imwrite("extraction.jpg", img2)
img2_plt = plt.imread('extraction.jpg')


fig, ax = plt.subplots(1,2)
ax[0].axis('off')
ax[1].axis('off')
ax[0].figure.set_size_inches(18,10)
ax[1].figure.set_size_inches(18,10)

ax[0].imshow(img_plt);
ax[1].imshow(img2_plt);

In [ ]:
# See the forefront image bigger
fig = plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(img2_plt)